In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache

from dotenv import load_dotenv
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, PromptTemplate

from langchain.prompts import MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

load_dotenv()

True

In [46]:
set_llm_cache(SQLiteCache("cache.db"))

chat = ChatOpenAI(
    temperature = 0.1,
)

memory = ConversationSummaryBufferMemory(
    llm = chat,
    max_token_limit = 80,
    return_messages = True
)

In [47]:
examples = [
    {
        "movie": "Wicked",
        "answer": "👠✨🧙‍♀️"
    },
    {
        "movie": "venom3",
        "answer": "🕷️🖤🔥"
    }
]

example_chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Express the movie {movie}"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_chat_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You speak with only 3 emojis."),
        example_prompt,
        # return_messages 를 받아 오는 데 몇개가 올지 모르는 상황
        MessagesPlaceholder(variable_name = "history"),
        ("human", "Express the movie {movie}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history = load_memory) | final_prompt | chat

def invoke_chain(question):
    result = chain.invoke({"movie": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [48]:
invoke_chain("Les Miserables")

content='🇫🇷⚔️🎶'


In [49]:
invoke_chain("Harry Potter and the Half Blood Prince")

content='⚡🔮🐍'


In [50]:
invoke_chain("The first movie I asked you.")

content='👠✨🧙\u200d♀️'
